![Matplotlib logo](img/matplotlib.svg)

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from src.training import *

# Basic Plotting

In this module, we will cover creation of a number of general plot types without spending much attention just yet on finer details of fonts, colors, styles, ancillary elements like axis labels, legends, and so on.  Be aware that all such things are configurable; but let us get the basic "ink on paper" first.

## Using Pandas

Since very often your data will be in Pandas DataFrames or Series, utilizing the Pandas API for accessing the underlying capabilities of Matplotlib is often a good first approach.  A nice thing about Pandas is that a simple, high-level call wraps numerous low-level decisions about ancillary decicions, usually producing "reasonable" choices (even if not finally optimal).

For a quick example, let us read in 20 century population estimates. One thing to notice is that that data is perhaps not in "tidy" format.  While the curent DataFrame has approximately the same number of rows as columns, in concept dates are more expandable and express a sequence rather than categories.

In [ ]:
population = pd.read_csv('data/population.csv', index_col='Region')
population

In [ ]:
population.sum()

We might visualize this using nothing more than the `.plot()` method (and a transpose of rows and columns, for reasons mentioned).

In [ ]:
population.T.plot();

Not bad as a one-line quick view.  Let us fine-tune it in a few ways, still within the Pandas API.  Notice the subtle use of $\LaTeX$ within textual elements, simply by placing it between $ signs.

In [ ]:
with plt.style.context('seaborn-v0_8'):
    population.T.plot(logy=True, figsize=(12, 4),
                      title="20 century population trend by region (log scale $10^n$)")

In [ ]:
with plt.style.context('dark_background'):
    population.T.plot(kind='area', figsize=(10, 4), 
                      title="Region component and total population over 20 centuries")

### Time series

Keeping in mind that **everything** Pandas does is wrap underlying capabilities in Matplotlib itself, it is worth nothing that one strong point in Pandas is timeseries handling.  The population information above was implicitly a timeseries, but we had not actually converted the dates to actual datetime values rather than strings.

If we work with with true datetime information, the Pandas wrapper handles "pretty" date tick labels for us, including decisions about the frequency to represent. In earlier versions Pandas would automatically sort a datetime index; in recent versions we need to explicitly sort this index.

In [ ]:
(pd.read_csv('data/verlegenhuken.csv', parse_dates=['DATE'])
     .set_index('DATE')
     .sort_index()  # This formerly happened implicitly for plots
     .loc[:, ['MIN', 'MAX']]
     .plot(figsize=(12, 3),
           title="Minimum and maximum daily ℉ in Verlegenhuken")
);

# Using Matplotlib

Some, but not all, of the options available for Matplotlib plots can be passed through to the Pandas API.  However, Matplotlib is perfectly happy to deal with any sequences of numbers, not only those in Pandas DataFrames or Series.  It works happily with NumPy arrays, with plain Python lists, and with a variety of other array and DataFrame libraries, such as [xarray](https://xarray.pydata.org/en/stable/).

Here are a sequence of numbers whose pattern is probably not evident on immediate visual inspection.  But if we graph them, a pattern easily emerges.

In [ ]:
nums = [+0.00, +0.12, +0.25, +0.37, +0.49, +0.59, +0.69, +0.78, +0.85, +0.91, 
        +0.95, +0.98, +0.99, +0.99, +0.97, +0.93, +0.88, +0.82, +0.74, +0.64, 
        +0.54, +0.43, +0.31, +0.19, +0.06, -0.06, -0.19, -0.31, -0.43, -0.54, 
        -0.64, -0.74, -0.82, -0.88, -0.93, -0.97, -0.99, -0.99, -0.98, -0.95, 
        -0.91, -0.85, -0.78, -0.69, -0.59, -0.49, -0.37, -0.25, -0.12, -0.01]

In [ ]:
plt.plot(nums);

## Some Footnotes

There are a few small technical details we have used but skipped over discussing so far.  Let us make them explicit.

* Within a Jupyter notebook, if you want plots to render inline, use the "magic" `%matplotlib inline` at some point.  There also exists a magic `%matplotlib notebook` that produces a slightly interactive view of plots, similar to the standalone viewer.

* You may have noticed an odd semi-colon at the end some cells.  This is a quick way to supress the value of the last line being echoed, where that object is some plot object, a list of them, or similar.  You can achieve the same effect also by assigning to a throwaway name: `_ = plt.plot(nums)`.

* Outside of Jupyter you can launch the viewer with `plt.show()`, which will launch something looking similar to this (if your tools/GUI are configured to allow this):

<img src="img/Matplotlib-gui.png" alt="Matplotlib GUI" width="50%"/>

* There is a simple procedural way of using Matplotlib that emulates MATLAB.  This style is generally discouraged.  At an intermediate level, we can call methods on the `plt` object that are equally procedural.  But what you should become comfortable with is the more object-oriented style in which you create figure and axis objects, manipulate legend objects, and ylabel objects, colormap objects, and so on.

The procedural style in a Python shell:

```python
>>> import numpy as np
>>> from pylab import *
>>> figure()
<Figure size 640x480 with 0 Axes>
>>> plot(np.sin(np.linspace(0, 2*np.pi, 50)))
[<matplotlib.lines.Line2D object at 0x7efcf3049350>]
>>> xlabel("Independent Variable")
Text(0.5, 0, 'Independent Variable')
>>> ylabel('Dependent Variable')
Text(0, 0.5, 'Dependent Variable')
>>> title("Representing a sin curve")
Text(0.5, 1.0, 'Representing a sin curve')
>>> show()
```

# Plot Types

Matplotlib comes with a huge number of plot types, all customizable in many details.  Let us show just a couple here beyond the basic line chart.  For example, bar charts are often useful for similar sequential information as line charts (where the number of samples is limited, e.g. fewer than 75 or so).

In [ ]:
# Illustrate the different long term trends in Europe vs Africa
e_asia = population.loc['East Asia']
s_asia = population.loc['South Asia']
world = population.sum()
x = range(len(e_asia))
# One technique is to juxtapose bars with different width
plt.bar(x, world, color="#B7B7F0", width=0.9, alpha=0.25)
plt.bar(x, s_asia, label="South Asia", width=0.6, color="lightblue")
plt.bar(x, e_asia, label="East Asia", width=0.2, color="green", alpha=0.75)
# Legend corresponds to labels given (can caption only some elements)
# E.g. world is backgrounded not captioned
plt.legend()
plt.yscale('log')
plt.xticks(x, world.index)
plt.title("Comparing Population Trends (millions)")

# We like this chart and want to save it
plt.savefig('tmp/population-trends.png', dpi=300);

## More Dimensions

Often we want to represent more than two dimensions of data.  A variety of techniques lend themselves to this; keep in mind the relative discernability of perceptual features mentioned in the prior module.  

The following is extracted from the NOAA temperature data for January 1.  Temperatures are normalized between 0 for coldest and 1 for warmest, simply to aid the following example.  One thing you will notice right off is that most latitude/longitude combinations do not have a station at that location.

In [ ]:
jan1_temps = pd.read_pickle('data/temp_grid.pkl')
jan1_temps.sort_index(ascending=False, inplace=True)
jan1_temps

A "heat map" can be very literal in this case, although the same style is used to represent many quantities.  There are definitely things we could cleanup in this data. For example, many latitudes and longitudes are missing entirely, hence the grid spacing is not spatially uniform.

In [ ]:
fig, ax = plt.subplots()
ax.imshow(jan1_temps.values, cmap='cool')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

Seaborn is a library that, like Pandas, lives on top of Matplotlib and provides a variety of statistical graphs.  It also pulls metadata from Pandas DataFrames where those are used.

In [ ]:
import seaborn as sns
sns.heatmap(jan1_temps, cmap='cool');

# Exercises

In the presentation of plotting with Pandas, three plots based on population data were shown.  Try to replicate each of those using only Matplotlib itself, and even using only the underlying NumPy arrays containing the data.  If you can improve on visual or arrangement elements of what Pandas does, that is even better.

In [ ]:
# Pull the arrays from the DataFrame, 
# then make plots based just on the underlying data
europe = population.loc['Europe'].values
europe
# ... other data elements ...
# ... make plots

Similarly to the above exercises, try to do the same pure-Matplotlib replication of the presentation of temperature data.

In [ ]:
minmax = (pd.read_csv('data/verlegenhuken.csv', parse_dates=['DATE'])
     .set_index('DATE')
     .loc[:, ['MIN', 'MAX']])
minmax.head()

In [ ]:
# Dates are not sorted on disk, we better fix that
minmax.sort_index(inplace=True)
dates = minmax.index.values
mintemp = minmax.MIN.values
maxtemp = minmax.MAX.values
dates[:3], mintemp[:3], maxtemp[:3]

In [ ]:
# Create a graph of minimum and maximum temperature by date from arrays
...